In [1]:
from imutils.video import VideoStream
import cv2
import imutils
import numpy as np
import time
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt
from os import listdir
from keras.preprocessing import image as ima
import csv
import pandas as pd

Using TensorFlow backend.


In [2]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

def loadVggFaceModel():
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Convolution2D(4096, (7, 7), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation('softmax'))
    model.load_weights('D:/python_AI/vgg_face_weights.h5')
    vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)
    return vgg_face_descriptor

In [3]:
def employee_database(employee_pictures,model):
    employees=dict()
    for file in listdir(employee_pictures):
        employee,extension=file.split(".")
        employees[employee]=model.predict(preprocess_image('{}/{}.{}'.format(employee_pictures,employee,extension)))[0,:]
    return employees

In [4]:
def findCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

In [5]:
vs = VideoStream(src=0).start()
model=loadVggFaceModel()
color = (0,255,0)
faceNet = cv2.dnn.readNetFromCaffe('face-detector-deploy.prototxt', 'res10_300x300_ssd_iter_140000.caffemodel')
employees=employee_database("D:/python_AI/face_test",model)
faceDetectionMeanPixelIntensityValues = (104.0, 177.0, 123.0)
data = pd.read_csv("D:/python_AI/employee_database.csv") 
while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=1000)
    (h, w) = frame.shape[:2]
    image = cv2.resize(frame, (300, 300))
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), faceDetectionMeanPixelIntensityValues)
    faceNet.setInput(blob)
    faceDetections = faceNet.forward()
    for i in range(0, faceDetections.shape[2]):
        faceDetectionConfidence = faceDetections[0, 0, i, 2]
        if faceDetectionConfidence < 0.55:
            continue
        box = faceDetections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        face = frame[startY:endY, startX:endX]
        x=startX
        y=startY
        if len(face) == 0 or len(face[0]) == 0:
            continue
        detected_face = cv2.resize(face, (224, 224))
        img_pixels = ima.img_to_array(detected_face)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels=preprocess_input(img_pixels)
        #img_pixels /= 255
        captured_representation = model.predict(img_pixels)[0,:]
        found=0
        #cv2.imshow('image',detected_face)
        for i in employees:
            employee_name = i
            representation = employees[i]
            similarity = findCosineSimilarity(representation, captured_representation)
            #print(similarity)
            if(similarity< 0.3):
                cv2.putText(frame, employee_name, (startX-220, startY+20),cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                boxColor = (0, 255, 255)
                cv2.rectangle(frame, (startX, startY), (endX, endY), boxColor, 2)
                if employee_name==data.Name.to_string(index=False):
                    cv2.putText(frame, data.Employer.to_string(index=False), (startX-220, startY+50),cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                    cv2.putText(frame, data.Location.to_string(index=False), (startX-220, startY+80),cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                    cv2.putText(frame, data.State.to_string(index=False), (startX-220, startY+110),cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                
    if True:
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
cv2.destroyAllWindows()
vs.stop()
exit()

In [11]:
employees=employee_database("D:/python_AI/face_test")

In [12]:
employees


{'Sri Harsha N': array([ 1.2818015 , -0.63073003,  0.4664738 , ..., -2.3125584 ,
         0.45995238,  1.9206898 ], dtype=float32)}

In [1]:
representation

NameError: name 'representation' is not defined

In [2]:
len(representation)

NameError: name 'representation' is not defined

In [1]:
a=[1,2,3,4]

In [2]:
a[1:2]

[2]

In [3]:
a[1:3]

[2, 3]

In [4]:
a[3:1]

[]

In [6]:
a=preprocess_image('D:/python_AI/face_test/Sri Harsha N.jpg')

In [7]:
a

array([[[[29.060997, 38.221   , 42.32    ],
         [29.060997, 38.221   , 42.32    ],
         [29.060997, 38.221   , 42.32    ],
         ...,
         [12.060997, 17.221   , 21.32    ],
         [10.060997, 18.221   , 19.32    ],
         [10.060997, 18.221   , 19.32    ]],

        [[29.060997, 38.221   , 42.32    ],
         [29.060997, 38.221   , 42.32    ],
         [29.060997, 38.221   , 42.32    ],
         ...,
         [12.060997, 17.221   , 21.32    ],
         [10.060997, 18.221   , 19.32    ],
         [10.060997, 18.221   , 19.32    ]],

        [[29.060997, 38.221   , 42.32    ],
         [29.060997, 38.221   , 42.32    ],
         [29.060997, 38.221   , 42.32    ],
         ...,
         [12.060997, 17.221   , 21.32    ],
         [10.060997, 18.221   , 19.32    ],
         [10.060997, 18.221   , 19.32    ]],

        ...,

        [[46.060997, 52.221   , 53.32    ],
         [46.060997, 52.221   , 53.32    ],
         [46.060997, 52.221   , 53.32    ],
         ...,


In [9]:
a.size

150528

In [10]:
a.shape

(1, 224, 224, 3)

In [11]:
np.amax(a)

131.32

In [1]:
import csv
import pandas as pd

In [5]:
employee_database=open("D:/python_AI/employee_database.csv", "r")

In [16]:
data = pd.read_csv("D:/python_AI/employee_database.csv") 

In [17]:
data

,Name,Employer,Location,State,LinkedIn
0,Sri Harsha N,Sallie Mae,Newark,Delaware,https://www.linkedin.com/in/sri-harsha-n-7415b...


In [27]:
print(data[['Name']].to_string(index=False))

Name
Sri Harsha N


In [33]:
'Sri Harsha N'==data.Name.to_string(index=False)

True